# How to Develop and Run the Tasks and Workflows Remotely via Jupiter Notebook

Before you start, make sure your flytekit version is higher than 1.14.0. If not, please upgrade it by running the following command:
```bash
!pip install flytekit==1.14.0
```

## 01 Setup the Interface to Connect to the Flyte Backend

Before starting to develop and run the tasks and workflows remotely via Jupiter Notebook, let's setup the interface via [flytekit.remote](https://docs.flyte.org/projects/flytekit/en/latest/design/control_plane.html) to interact with the Flyte backend. 

**Make sure to set `interactive_mode_enabled` to `True` to support auto pickling and registering entities to backend from Jupyter Notebook environment.**

In [1]:
from flytekit.remote import FlyteRemote
from flytekit.configuration import Config

# The `for_sandbox` method instantiates a connection to the demo cluster.
remote = FlyteRemote(
    Config.for_sandbox(), 
    default_project="flytesnacks", 
    default_domain="development", 
    interactive_mode_enabled=True,
)

23:40:27.971008 WARNING  remote.py:230 - Jupyter notebook and interactive task  
                         support is still alpha.

## 02 Develop the tasks & workflows

Let's create a simple task and workflow to demonstrate how to develop and run the tasks and workflows remotely via Jupiter Notebook.

In [2]:
from flytekit import task, workflow

@task
def hello(name: str) -> str:
    return f"Hello {name}!"

@task
def world(pre: str) -> str:
    return f"{pre} Welcome to the Jupyter Notebook!"

@workflow
def wf(name: str) -> str:
    return world(pre=hello(name=name))

When executing the task or workflow via `FlyteRemote` interface, the related entities will be automatically registered to the Flyte backend. You can navigate to the Flyte Console via the URL to see the progress and outputs of the executions, or you can use `remote.wait()` to wait for the execution to complete. This may take a few seconds to minutes depending on the size of the task and workflow.

In [3]:
# Execute the task
out = remote.execute(hello, inputs={"name": "Flyte"})

# This will print the URL to the console
print(remote.generate_console_url(out))

# Wait for the task to complete
out = remote.wait(out)

# Print the outputs
print(out.outputs)

http://localhost:30080/console/projects/flytesnacks/domains/development/executions/alnrr5cmh4gbrbfcvp8k
{'o0': 'Hello Flyte!'}


If you are not interested in the URL to the Flyte Console and want the execution wait directly to complete, you can set `wait=True` in the `remote.execute()` method.

In [4]:
# Execute the workflow and wait for it to complete
out = remote.execute(hello, inputs={"name": "world"}, wait=True)

# Print the outputs
print(out.outputs)

{'o0': 'Hello world!'}


Similarly, we can also execute the workflows remotely and extract the outputs.

In [5]:
# Execute the workflow and wait for it to complete
out = remote.execute(wf, inputs={"name": "world"}, wait=True)

# Print the outputs
print(out.outputs)

{'o0': 'Hello world! Welcome to the Jupyter Notebook!'}


If you repeat the execution, it will not re-register the entities to the backend, instead it will reuse the existing entities. However, if you update and run the cells that define the tasks or workflows, it will register the updated entities to the backend when you execute them.

Feel free to update the tasks and workflows definition and see how it works!

## 03 Develop the Workflow Contains `map_task`

`map_task` is also supported in the remote execution. You can define the `map_task` in the workflow and execute it remotely.

In [6]:
from flytekit import task, workflow, map_task
from functools import partial

@task
def fn(x: int, y: int) -> int:
    return x + y + 2

@workflow
def workflow_with_maptask(data: list[int], y: int) -> list[int]:
    partial_fn = partial(fn, y=y)
    return map_task(partial_fn)(x=data)

Let's execute the workflow remotely and extract the outputs.

In [7]:
# Execute the workflow with map_task and wait for it to complete
out = remote.execute(workflow_with_maptask, inputs={"data": [1, 2, 3], "y": 4}, wait=True)

# Print the outputs
print(out.outputs)

{'o0': [7, 8, 9]}
